In [5]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error

from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tzfpy import get_tz
from pytz import timezone
import kds
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import ydata_profiling as df_report

import seaborn as sns
import plotly.express as px
import matplotlib as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import sweetviz as sv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h3
import keplergl



### Load data from private dataset and public dataset

In [6]:
df_usage = pd.read_csv('data/usage_data_2023_05_01_to_15.csv')
display(df_usage.head(2))
df_airtraffic_trends = pd.read_csv('data/Airline_Transportation_Data.csv')
display(df_airtraffic_trends.head(2))

,AIRLINE,TAIL_ID,USAGE_TSTAMP,PREMIUM_PRODUCT,ESI_PRODUCT,TMO_PRODUCT,BASIC_PRODUCT,OTHER_PRODUCT,DOOR_CLOSE_TSTAMP,DOOR_OPEN_PLUS15_TSTAMP,TIME_INTO_FLIGHT,FLIGHT_PHASE,LATITUDE,LONGITUDE,ALTITUDE,TOTAL_FL_MBPS,TOTAL_RL_MBPS,SEAT_CAPACITY,ECO_SEATS
0,AAL,N314PD,2023-05-04T01:20:00Z,1,0,0,0,1,2023-05-04T00:00:52.341Z,2023-05-04T02:04:59.387Z,80.0,10k,27.33,-80.38,29175.63,6.93,1.16,172.0,114.0
1,DAL,N384DA,2023-05-04T01:20:00Z,1,0,0,0,1,2023-05-03T23:09:20.904Z,2023-05-04T03:30:35.561Z,131.0,10k,55.44,-133.12,38696.88,4.64,1.95,160.0,160.0


,ID,OBS_DATE,ASM_US_D_I,ASM_D,ASM_I,LOAD_FACTOR_D_PCT_US,LOAD_FACTOR_I_PCT_US
0,SATD200001,1/1/00,76146639.0,56288872.0,19857767.0,62.0,67.2
1,SATD200002,2/1/00,73011377.0,54329819.0,18681558.0,66.3,67.3


### Performing column re-names, EDA,Data Cleaning, Feature Engineering and Feature Selection 

In [7]:

df_airtraffic_trends.rename(columns={'OBS_DATE': 'Month','ASM_US_D_I':'Aggr_seat_miles_domestic_and_international','LOAD_FACTOR_D_PCT_US':'Load_factor_domestic_pct_us'}, inplace=True)
df_airtraffic_trends.drop(['ID','ASM_D','ASM_I','LOAD_FACTOR_I_PCT_US'], axis=1, inplace=True)
df_airtraffic_trends['Month'] = pd.to_datetime(df_airtraffic_trends['Month'])
df_usage.columns = df_usage.columns.str.lower()
df_airtraffic_trends.columns = df_airtraffic_trends.columns.str.lower()
df_usage['usage_tstamp'] = pd.to_datetime(df_usage['usage_tstamp'])
df_usage['door_close_tstamp'] = pd.to_datetime(df_usage['door_close_tstamp'])
df_usage['door_open_plus15_tstamp'] = pd.to_datetime(df_usage['door_open_plus15_tstamp'])

# Based on several iterations of the EDA, it felt like I need to come up with a new feature which is percentage of flight completed 
# and need to find all time related variables in local time than in utc. Used lat,lon to convert time to local timezone.
# Dropped all rows with null vallues for lat,long

df_usage['pct_flight_completed'] = df_usage['time_into_flight']/((df_usage['door_open_plus15_tstamp'] - df_usage['door_close_tstamp']).dt.total_seconds() / 60)
df_usage.drop(['tail_id','door_close_tstamp','door_open_plus15_tstamp'], axis=1, inplace=True)
df_usage.dropna(subset=['latitude','longitude','seat_capacity','eco_seats','airline'],inplace=True)
df_usage['free_model'] = df_usage['airline'].apply(lambda x: 1 if x in ['DAL','JBU','JBU-R','QFA'] else 0)
# The initial data set had a lot of datapoints, so I sampled 1% of the data to do the EDA and build the model
df_usage_sampled = df_usage.sample(frac=0.05, random_state=42,replace=False)

display(df_airtraffic_trends.head(2))
display(df_usage_sampled.head(2))

,month,aggr_seat_miles_domestic_and_international,load_factor_domestic_pct_us
0,2000-01-01,76146639.0,62.0
1,2000-02-01,73011377.0,66.3


,airline,usage_tstamp,premium_product,esi_product,tmo_product,basic_product,other_product,time_into_flight,flight_phase,latitude,longitude,altitude,total_fl_mbps,total_rl_mbps,seat_capacity,eco_seats,pct_flight_completed,free_model
201556,DAL,2023-05-10 14:45:00+00:00,1,0,0,0,1,88.0,10k,41.29,-113.40,35214.50,22.53,4.15,180.0,160.0,0.375718,1
2269270,UAL-R,2023-05-06 13:05:00+00:00,0,0,1,1,1,74.0,10k,39.22,-111.05,38231.88,1.01,0.14,166.0,166.0,0.393268,0


In [8]:
# Performing timezone trasnlations to get local time as well as hour of day and day of week

df_usage_sampled['usage_utc_hour'] = df_usage_sampled['usage_tstamp'].dt.hour
df_usage_sampled['usage_month']=df_usage_sampled['usage_tstamp'].dt.month_name()
df_usage_sampled['usage_utc_day']=df_usage_sampled['usage_tstamp'].dt.day_name()

def local_time_mapper(x):
    return get_tz(lat=x['latitude'], lng=x['longitude'])


df_usage_sampled['usage_tz'] = df_usage_sampled.apply(lambda x: local_time_mapper(x), axis=1)

# Throwing away few rows which had no timezone look up, very small numebr of rows
df_usage_sampled = df_usage_sampled[df_usage_sampled['usage_tz']!='']

# Converting utc time to local time based on the timezone and then getting the local hour of the day and day of the week

df_usage_sampled['usage_local_hour'] = df_usage_sampled.apply(lambda row: row['usage_tstamp'].tz_convert(timezone(row['usage_tz'])).hour, axis=1)
df_usage_sampled['usage_local_hour_shifted'] = df_usage_sampled['usage_local_hour'].apply(lambda x: (pd.Timedelta(hours=x) + pd.Timedelta(hours=-3)).components.hours)
df_usage_sampled['usage_local_day'] = df_usage_sampled.apply(lambda row: row['usage_tstamp'].tz_convert(timezone(row['usage_tz'])).strftime("%A"), axis=1)

display(df_usage_sampled.head(2))

,airline,usage_tstamp,premium_product,esi_product,tmo_product,basic_product,other_product,time_into_flight,flight_phase,latitude,...,eco_seats,pct_flight_completed,free_model,usage_utc_hour,usage_month,usage_utc_day,usage_tz,usage_local_hour,usage_local_hour_shifted,usage_local_day
201556,DAL,2023-05-10 14:45:00+00:00,1,0,0,0,1,88.0,10k,41.29,...,160.0,0.375718,1,14,May,Wednesday,America/Denver,8,5,Wednesday
2269270,UAL-R,2023-05-06 13:05:00+00:00,0,0,1,1,1,74.0,10k,39.22,...,166.0,0.393268,0,13,May,Saturday,America/Denver,7,4,Saturday


In [9]:
# Dropping un-wanted rows before doing one more final round of eda through sweetviz

df_usage_sampled.drop(['usage_tstamp','usage_tz'], axis=1, inplace=True)
# Doing sv analyze for all features but airline to mask out customer information from public view


df_usage_eda_report = sv.analyze(df_usage_sampled[[col for col in df_usage_sampled.columns if col != 'airline']], target_feat='total_fl_mbps')
df_usage_eda_report.show_html('images/eda_report.html')


                                             |          | [  0%]   00:00 -> (? left)

Report images/eda_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [10]:
# Based on the output of the FInal run of eda, dropping unwanted columns as well as columns which are not available at the time of prediction ('total_rl_mbps')
df_usage_sampled.drop(columns=['usage_utc_day','usage_utc_hour','usage_local_hour','usage_month','total_rl_mbps'],inplace=True)
df_usage_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120567 entries, 201556 to 2510421
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   airline                   120567 non-null  object 
 1   premium_product           120567 non-null  int64  
 2   esi_product               120567 non-null  int64  
 3   tmo_product               120567 non-null  int64  
 4   basic_product             120567 non-null  int64  
 5   other_product             120567 non-null  int64  
 6   time_into_flight          120567 non-null  float64
 7   flight_phase              120567 non-null  object 
 8   latitude                  120567 non-null  float64
 9   longitude                 120567 non-null  float64
 10  altitude                  120567 non-null  float64
 11  total_fl_mbps             120567 non-null  float64
 12  seat_capacity             120567 non-null  float64
 13  eco_seats                 120567 non-n

In [11]:
# Converting local_hour and pct flight completed to categorical variables based on the output of final eda run
df_usage_sampled['usage_local_hour_shifted'] = df_usage_sampled['usage_local_hour_shifted'].astype('str')
df_usage_sampled['pct_flight_completed'] = round((100*df_usage_sampled['pct_flight_completed']/10),0).astype('str')

In [12]:
# plot a cdf to see the outliers using plotly express for target variable total_fl_mbps
fig = px.histogram(df_usage_sampled, x="total_fl_mbps", nbins=500, title='CDF of total_fl_mbps',cumulative=True, histnorm='probability density')
fig.show()
# fig.write_image("images/cdf_total_fl_mbps.png")

#Throwing outliers in total_fl_mbps which are between 0.1 and  95th percentile
print('After removing outliers in total_fl_mbps, the shape of the cdf is')
df_usage_sampled = df_usage_sampled[ ( df_usage_sampled['total_fl_mbps']<df_usage_sampled['total_fl_mbps'].quantile(0.95)) & (df_usage_sampled['total_fl_mbps']>df_usage_sampled['total_fl_mbps'].quantile(0.001)) ]
fig1 = px.histogram(df_usage_sampled, x="total_fl_mbps", nbins=500, title='CDF of total_fl_mbps',cumulative=True, histnorm='probability density')
fig1.show()

After removing outliers in total_fl_mbps, the shape of the cdf is


### Performing column transformations for categorical and numeric featues

In [13]:
# Applying column transformer to encode categorical variables and scale numeric variables

# Get columns of integer or float type
numeric_cols = df_usage_sampled.select_dtypes(include=['int64', 'float64']).columns
print(numeric_cols)
# Get columns of object type
object_cols = df_usage_sampled.select_dtypes(include=['object']).columns
print(object_cols)

# Categorical feature columns to one-hot encode
categorical_cols = ['airline','flight_phase','usage_local_day','pct_flight_completed','usage_local_hour_shifted','pct_flight_completed']
# Numerical feature columns to standard scale
numerical_cols = ['altitude','seat_capacity','eco_seats']

# Create transformers for one-hot encoding and standard scaling
column_trans = ColumnTransformer(
    transformers=[
        ('ohe',OneHotEncoder(sparse=False), categorical_cols),
        ('std_scaled',StandardScaler(), numerical_cols),
    ],
    remainder='passthrough',
    )
column_trans.set_output(transform='pandas')
transformed_data = column_trans.fit_transform(df_usage_sampled)
df_usage_column_transformed = pd.DataFrame(transformed_data)

df_usage_column_transformed.rename(columns={'remainder__total_fl_mbps':'total_fl_mbps'}, inplace=True)
display(df_usage_column_transformed.head(2))

Index(['premium_product', 'esi_product', 'tmo_product', 'basic_product',
       'other_product', 'time_into_flight', 'latitude', 'longitude',
       'altitude', 'total_fl_mbps', 'seat_capacity', 'eco_seats',
       'free_model'],
      dtype='object')
Index(['airline', 'flight_phase', 'pct_flight_completed',
       'usage_local_hour_shifted', 'usage_local_day'],
      dtype='object')


,ohe__airline_AAL,ohe__airline_AMX,ohe__airline_AZU,ohe__airline_DAL,ohe__airline_DJT,ohe__airline_ELY,ohe__airline_FIN,ohe__airline_ICE,ohe__airline_JBU,ohe__airline_JBU-R,...,remainder__premium_product,remainder__esi_product,remainder__tmo_product,remainder__basic_product,remainder__other_product,remainder__time_into_flight,remainder__latitude,remainder__longitude,total_fl_mbps,remainder__free_model
201556,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,88.0,41.29,-113.40,22.53,1
2269270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,1,74.0,39.22,-111.05,1.01,0


### Iterating over different ML Algorithms and  grid searching for hyper parameters within each

In [14]:
# Split the df into train test split with 80:20 ratio 

X = df_usage_column_transformed.loc[:, df_usage_column_transformed.columns != 'total_fl_mbps']
y = df_usage_column_transformed['total_fl_mbps']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining a set of regression models to be used for hyper parameter tuning

regressors = {
    'Linear Regression': (LinearRegression(), {}),
    'Ridge Regression': (Ridge(fit_intercept=False), {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}),
    'Lasso Regression': (Lasso(), {'alpha': [0.00001,0.0001,0.001,0.01, 0.1, 1.0, 10.0, 100.0]}),
    'Random Forest Regressor': (RandomForestRegressor(n_jobs=-1), {'n_estimators': [ 50, 100, 200], 'max_depth': [5, 7, 10, 15, 20, 40]}),
    'AdaBoost Regressor': (AdaBoostRegressor(), {'n_estimators': [20, 50, 100, 200], 'learning_rate': [0.01, 0.1, 1 ,5]}),
    #'Support Vector Regressor': (SVR(), {'C': [1, 10], 'kernel': ['linear', 'rbf']})
}

best_models = []
# Perform grid search for each regressor
for regressor_name, (regressor, param_grid) in regressors.items():
    grid_search = GridSearchCV(regressor, param_grid, scoring='r2', cv=5, verbose=2)
    grid_search.fit(X_train, y_train)

    # Get the best estimator and make predictions
    best_regressor = grid_search.best_estimator_
    y_pred = best_regressor.predict(X_test)

    # Calculate and print the R2 score

    r2 = r2_score(y_test, y_pred)
    print(f"{regressor_name}: Best Parameters: {grid_search.best_params_}, R2 Score: {r2}")
    best_models.append({'regressor':regressor_name,'model':best_regressor})


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
Linear Regression: Best Parameters: {}, R2 Score: 0.4249325313413552
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ........................................alpha=0.001; total time=   0.1s
[CV] END ........................................alpha=0.001; total time=   0.1s
[CV] END ........................................alpha=0.001; total time=   0.1s
[CV] END ........................................alpha=0.001; total time=   0.1s
[CV] END ........................................alpha=0.001; total time=   0.1s
[C

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.465e+05, tolerance: 2.629e+02



[CV] END ........................................alpha=1e-05; total time=   6.1s


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.425e+05, tolerance: 2.616e+02



[CV] END ........................................alpha=1e-05; total time=   6.1s


...

[CV] END ........................................alpha=0.001; total time=   4.0s


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.977e+05, tolerance: 2.622e+02



[CV] END ........................................alpha=0.001; total time=   4.0s
[CV] END .........................................alpha=0.01; total time=   1.2s
[CV] END .........................................alpha=0.01; total time=   1.3s
[CV] END .........................................alpha=0.01; total time=   1.3s
[CV] END .........................................alpha=0.01; total time=   1.3s
[CV] END .........................................alpha=0.01; total time=   1.3s
[CV] END ..........................................alpha=0.1; total time=   0.3s
[CV] END ..........................................alpha=0.1; total time=   0.2s
[CV] END ..........................................alpha=0.1; total time=   0.2s
[CV] END ..........................................alpha=0.1; total time=   0.2s
[CV] END ..........................................alpha=0.1; total time=   0.2s
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ...................

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.316e+05, tolerance: 3.275e+02



[CV] END .......................max_depth=5, n_estimators=50; total time=   9.0s
[CV] END .......................max_depth=5, n_estimators=50; total time=   2.5s
[CV] END .......................max_depth=5, n_estimators=50; total time=   2.4s
[CV] END .......................max_depth=5, n_estimators=50; total time=   2.5s
[CV] END .......................max_depth=5, n_estimators=50; total time=   2.4s
[CV] END ......................max_depth=5, n_estimators=100; total time=   4.4s
[CV] END ......................max_depth=5, n_estimators=100; total time=   4.4s
[CV] END ......................max_depth=5, n_estimators=100; total time=   4.3s
[CV] END ......................max_depth=5, n_estimators=100; total time=   4.4s
[CV] END ......................max_depth=5, n_estimators=100; total time=   4.4s
[CV] END ......................max_depth=5, n_estimators=200; total time=   8.6s
[CV] END ......................max_depth=5, n_estimators=200; total time=   8.4s
[CV] END ...................

In [15]:
display(best_models[3]['regressor'])
display(best_models[3]['model'].feature_importances_)
importances = best_models[3]['model'].feature_importances_

'Random Forest Regressor'

array([5.51065049e-03, 1.56940018e-03, 5.53160294e-05, 6.04743137e-04,
       7.65561120e-06, 4.31167368e-05, 3.67414625e-08, 2.10912121e-05,
       9.75784987e-03, 3.20563627e-03, 4.87698039e-06, 4.98531416e-04,
       4.23347898e-06, 2.84905500e-03, 9.49063047e-05, 2.09000772e-05,
       2.69545711e-04, 7.39774868e-04, 3.92485819e-04, 1.29523012e-05,
       1.30830985e-03, 1.36148783e-01, 1.28820115e-03, 6.70693664e-03,
       7.31053158e-03, 9.25764566e-03, 5.72659021e-03, 7.14656065e-03,
       6.97824140e-03, 7.09481081e-03, 2.47519274e-04, 1.29747164e-03,
       5.10361389e-03, 1.90877130e-03, 2.19609904e-03, 2.22277546e-03,
       2.22112525e-03, 2.20177003e-03, 1.90943668e-03, 1.48452853e-03,
       1.56821034e-03, 5.31584566e-03, 5.61206423e-03, 5.84976062e-03,
       5.11579336e-03, 4.89680053e-03, 5.12575114e-03, 5.08584839e-03,
       4.70029695e-03, 4.43336616e-03, 4.38666320e-03, 3.16341845e-03,
       3.12193438e-03, 5.26809703e-03, 3.60670359e-03, 3.63812145e-03,
      

In [ ]:
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]
sorted_features = X_test.columns[indices]

# Plot the feature importances
plt.figure(figsize=(8, 4))
plt.title("Feature Importance")
plt.bar(range(X_test.shape[1]), sorted_importances, align="center")
plt.xticks(range(X_test.shape[1]), sorted_features, rotation=90)
plt.tight_layout()
plt.show()

In [ ]:

# Print mean absolute percentage error (MAPE), R2 score, and mean absolute error (MAE)


best_regressor = best_models[3]['model']
y_pred = best_regressor.predict(X_test)

# Calculate and print the MAPE score
mape = round(mean_absolute_percentage_error(y_test, y_pred),2)
mae = round(mean_absolute_error(y_test, y_pred),2)
mse = round(mean_squared_error(y_test, y_pred),2)
r2 = r2_score(y_test, y_pred)
print("Best Regressor's Scores: MAPE : {mape}, MAE : {mae}, MSE : {mse}, R2 : {r2}".format(mape=mape,mse=mse,mae=mae,r2=r2))

In [ ]:
# Plot the predicted vs actual values
plt.figure(figsize=(10, 6))
plt.title("Predicted vs Actual")
plt.scatter(y_test, y_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()
